In [43]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
from sklearn.model_selection import StratifiedKFold


import xgboost as xg

from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_validate

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/water-pollution/water pollution (data for classification models)/Submission.csv
/kaggle/input/water-pollution/water pollution (data for classification models)/Target.csv
/kaggle/input/water-pollution/water pollution (data for classification models)/Train.csv
/kaggle/input/water-pollution/water pollution (data for classification models)/Test.csv
/kaggle/input/water-pollution/water_pollution_dataset/Submission.csv
/kaggle/input/water-pollution/water_pollution_dataset/Target.csv
/kaggle/input/water-pollution/water_pollution_dataset/Train.csv
/kaggle/input/water-pollution/water_pollution_dataset/Test.csv


In [44]:
DF_train = pd.read_csv('/kaggle/input/water-pollution/water pollution (data for classification models)/Train.csv', delimiter = ',')
DF_test = pd.read_csv('/kaggle/input/water-pollution/water pollution (data for classification models)/Test.csv', delimiter = ',')
Target = pd.read_csv('/kaggle/input/water-pollution/water pollution (data for classification models)/Target.csv', delimiter = ',')
Submission = pd.read_csv('/kaggle/input/water-pollution/water pollution (data for classification models)/Submission.csv', delimiter = ',')

### Concatinate train and test

In [45]:
df = pd.concat([DF_train, DF_test])

## View categorical and numerical columns


In [46]:
cat_columns = []
num_columns = []

for column_name in df.columns:
    if (df[column_name].dtypes == object):
        cat_columns +=[column_name]
    else:
        num_columns +=[column_name]

print('categorical columns:\t ',cat_columns, '\n len = ',len(cat_columns))

print('numerical columns:\t ',  num_columns, '\n len = ',len(num_columns))

categorical columns:	  ['code', 'period', 'id', 'Country'] 
 len =  4
numerical columns:	  ['Unnamed: 0', 'year', 'tourists', 'venue', 'rate', 'food', 'glass', 'metal', 'other', 'paper', 'plastic', 'leather', 'green_waste', 'waste_recycling'] 
 len =  14


### Move 'year' to num columns


In [47]:
cat_columns = ['code', 'period', 'id', 'Country', 'year']
num_columns = ['tourists', 'venue', 'rate', 'food', 'glass', 'metal', 'other', 'paper', 'plastic', 'leather', 'green_waste', 'waste_recycling'] 

# OHE

In [48]:
df_ohe = pd.get_dummies(df[cat_columns])

In [49]:
df_all = pd.concat([df_ohe, df[num_columns]], axis=1)

# Train test split

In [50]:
# Creating StandardScaler Object

scaler = StandardScaler()
df_scale = pd.DataFrame(scaler.fit_transform(df_all[num_columns]))
df_all = pd.concat([df_ohe.reset_index(drop=True), df_scale.reset_index(drop=True)], axis=1)

In [51]:
len_test = len(DF_test)
train = df_all.iloc[:-len_test]
test = df_all.iloc[-len_test:]
print(DF_train.shape, train.shape, DF_test.shape, test.shape)

(2928, 18) (2928, 1674) (732, 18) (732, 1674)


In [52]:
X, y = pd.DataFrame(train).values, Target['polution_clf'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train.shape, X_test.shape, y_train.shape, y_test.shape

((2342, 1674), (586, 1674), (2342,), (586,))

# LogisticRegression

In [53]:
model_LogReg = LogisticRegression(penalty="none", 
                                  dual=False, 
                                  tol=1e-4, C=1.0, 
                                  fit_intercept=True, 
                                  intercept_scaling=1, 
                                  class_weight=None, 
                                  random_state=None, 
                                  solver="lbfgs", 
                                  max_iter=10000, 
                                  multi_class="auto", 
                                  verbose=0, 
                                  warm_start=False, 
                                  n_jobs=None, 
                                  l1_ratio=None)

model_LogReg.fit(X_train, y_train)


LogisticRegression(max_iter=10000, penalty='none')

In [54]:
y_predict = model_LogReg.predict(test)
Submission.polution_clf = y_predict

/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,


In [55]:
y_predict.shape, test.shape

((732,), (732, 1674))

# Cross val

In [56]:
scoring = {'ACC': 'accuracy',
           'F1': 'f1',
           'Precision': 'precision',
           'Recall': 'recall'}
           
scores = cross_validate(model_LogReg, X_train, y_train,
                      scoring=scoring, cv=StratifiedKFold(n_splits=5, shuffle = True, random_state=42))


print('Результаты Кросс-валидации')
DF_cv_logreg = pd.DataFrame(scores)
display(DF_cv_logreg)
print('\n')
print(DF_cv_logreg.mean()[2:])

Результаты Кросс-валидации


,fit_time,score_time,test_ACC,test_F1,test_Precision,test_Recall
0,12.666306,0.007316,0.795309,0.806452,0.781250,0.833333
1,15.512065,0.007423,0.761194,0.766667,0.766667,0.766667
2,18.806388,0.007770,0.747863,0.741228,0.782407,0.704167
3,16.849026,0.007828,0.756410,0.754310,0.781250,0.729167
4,17.669835,0.007064,0.758547,0.770791,0.750988,0.791667




test_ACC          0.763865
test_F1           0.767890
test_Precision    0.772512
test_Recall       0.765000
dtype: float64


In [57]:
scoring = {'ACC': 'accuracy',
           'F1': 'f1',
           'Precision': 'precision',
           'Recall': 'recall'}
           
scores = cross_validate(model_LogReg, test, y_predict,
                      scoring=scoring, cv=StratifiedKFold(n_splits=5, shuffle = True, random_state=42))


print('Результаты Кросс-валидации')
DF_cv_logreg = pd.DataFrame(scores)
display(DF_cv_logreg)
print('\n')
print(DF_cv_logreg.mean()[2:])

/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,
/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,
/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,
/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,
/opt/conda/lib/python3.7/site-packages/sklea

Результаты Кросс-валидации


/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,


,fit_time,score_time,test_ACC,test_F1,test_Precision,test_Recall
0,1.614296,0.038550,0.877551,0.886076,0.897436,0.875000
1,1.386049,0.041373,0.823129,0.843373,0.813953,0.875000
2,1.813215,0.037379,0.801370,0.802721,0.867647,0.746835
3,1.296411,0.037654,0.787671,0.812121,0.779070,0.848101
4,1.592807,0.038933,0.904110,0.914634,0.892857,0.937500




test_ACC          0.838766
test_F1           0.851785
test_Precision    0.850193
test_Recall       0.856487
dtype: float64


In [58]:

#print('F1:', f1_score(y_test, y_predict['polution_clf']))


In [59]:
#print(classification_report(y_test, y_predict, target_names=['polution_clf']))

## Sumbission results

In [66]:
! rm '/kaggle/working/water-pollution/water pollution (data for classification models)/Submission.csv'
! mkdir '/kaggle/working/water-pollution/'

rm: cannot remove '/kaggle/working/water-pollution/water pollution (data for classification models)/Submission.csv': No such file or directory


In [67]:
Submission.to_csv('/kaggle/working/water-pollution/Submission.csv',index=False)

In [68]:
! mkdir ~/.kaggle
! touch ~/.kaggle/kaggle.json
! echo '{"username":"hydramst","key": "" }' > ~/.kaggle/kaggle.json
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
cat ~/.kaggle/kaggle.json

In [70]:
#! kaggle competitions submit -c sf-matml-2022-classification -f '/kaggle/working/water-pollution/Submission.csv' -m "Message"

100%|██████████████████████████████████████| 5.30k/5.30k [00:02<00:00, 1.83kB/s]
Successfully submitted to SF_MatML_2022_Classification